<a href="https://colab.research.google.com/github/piratesdragon/homework_actual_problems/blob/main/homework_PEFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 1.

Дообучите языковую модель на датасете инструкций, используя LoRA. Проверьте, что дообученная модель отличается от изначальной - сгенерируйте продолжения для одних и тех же промптов и сравните результаты.

Вы можете взять за основу код семинара PEFT, изменив датасет цитат на датасет инструкций (можно просто скопировать из семинара про General_instruct_fine-tuning). 
Можно использовать alpaca_dataset, датасет Dolly 2 или ваш собственный переведенный датасет (или все вместе). 
Важно использовать модель с большим количеством параметров (относительно семинара по General instruct fine-tuning). Например, можно попробовать OPT-1.3b,OPT-2.7b или ai-forever/rugpt3large_based_on_gpt2.



In [26]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [27]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b", 
    load_in_8bit=True, 
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [28]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

In [29]:
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [30]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [31]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=32,
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6291456 || all params: 1322049536 || trainable%: 0.47588655558516074


### Датасет

In [32]:
!wget https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

data = json.load(open('alpaca_data.json'))

--2023-05-06 07:51:26--  https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22773992 (22M) [text/plain]
Saving to: ‘alpaca_data.json.3’

alpaca_data.json.3  100%[===================>]  21.72M   121MB/s    in 0.2s    

2023-05-06 07:51:26 (121 MB/s) - ‘alpaca_data.json.3’ saved [22773992/22773992]



In [33]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = json.load(open(data_path))

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [38]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "facebook/opt-1.3b",
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [41]:
trainer = transformers.Trainer(
    model=model, 
    train_dataset= SupervisedDataset(tokenizer=tokenizer, data_path="alpaca_data.json"),
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=400, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=1, 
        output_dir='outputs'
    ),
    data_collator=DataCollatorForSupervisedDataset(tokenizer=tokenizer)
)

In [42]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
1,2.194700
2,2.275200
3,1.798600
4,2.061400
5,2.004800
6,1.924100
7,2.107100
8,1.981700
9,2.264800
10,2.078500


TrainOutput(global_step=400, training_loss=1.705285763144493, metrics={'train_runtime': 1881.9449, 'train_samples_per_second': 3.401, 'train_steps_per_second': 0.213, 'total_flos': 8169890551726080.0, 'train_loss': 1.705285763144493, 'epoch': 0.12})

In [43]:
model.save_pretrained('opt_1.3_lora')

### Проверка на сгенерированных примерах

In [1]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "opt_1.3_lora"

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="facebook/opt-1.3b", 
                                             return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [2]:
def generate(text, tokenizer, model):
  batch = tokenizer(text, return_tensors='pt').to('cuda')
  output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.0, no_repeat_ngram_size=2)

  return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

### Примеры без дообучения

In [3]:
generate("Give three tips for passing an exam.",  tokenizer, model)

'Give three tips for passing an exam.\n\n1. Don’t be afraid to ask questions. If you don‘t know something, ask. You‚ll learn a lot. And you„ll be able to pass the exam with flying colors.2.'

In [4]:
generate("Who is the author of War and Peace?",  tokenizer, model)

"Who is the author of War and Peace?\n\nThe author is Leo Tolstoy.\nI'm not sure if you're asking about the book or the movie. The book is a novel, and the film is based on it."

In [5]:
generate("Name five things that you need to take on holiday",  tokenizer, model)

'Name five things that you need to take on holiday.\n\n1. A good book. I’m not talking about a book that”s on the beach, or in the library. You need a good, solid book to read. Something that will keep you entertained, and keep'

In [6]:
generate("Generate a question that could follow the following response: I don't like reading.",  tokenizer, model)

"Generate a question that could follow the following response: I don't like reading. I like to listen to music.\n\nThe question is: What is the best way to learn to read?\nAnswer: Read. Read a lot. Listen to a book. Watch a movie. Play a game. Talk to someone who reads"

In [7]:
generate("Recommend me a good science fiction movie.",  tokenizer, model)

'Recommend me a good science fiction movie.\nThe Martian.'

In [8]:
generate("Recommend me a good science fiction movie and tell me about it.",  tokenizer, model)

"Recommend me a good science fiction movie and tell me about it.\nThe Martian.   It's a great movie, but it's not a science-fiction movie."

### Примеры с дообучением

In [9]:
model = PeftModel.from_pretrained(model, peft_model_id)


In [10]:
generate("Give three tips for passing an exam.",  tokenizer, model)

'Give three tips for passing an exam.\n\n1. Know the material. The exam is not a guessing game. You should know the content of the exam and be able to answer questions about it. If you don’t know what the questions are about, ask your teacher or'

In [11]:
generate("Who is the author of War and Peace?",  tokenizer, model)

'Who is the author of War and Peace?\n\nThe author is Leo Tolstoy.\nWar and peace is a novel by Russian author Leo T. TolSTOY. It was first published in 1855 and is considered one of the greatest works of literature in the world.'

In [12]:
generate("Name five things that you need to take on holiday",  tokenizer, model)

'Name five things that you need to take on holiday.\n\n1. A good book. I love reading and I always have a book with me on the plane. It’s a great way to pass the time and it‘s also a good way of learning new things.'

In [13]:
generate("Generate a question that could follow the following response: I don't like reading.",  tokenizer, model)

"Generate a question that could follow the following response: I don't like reading. What is the best way to get me to read?\n\nAnswer: The best method to encourage reading is to make reading fun. Make reading a game, and make it a reward for good behavior. For example, if you read a book,"

In [14]:
generate("Recommend me a good science fiction movie.",  tokenizer, model)

'Recommend me a good science fiction movie.\nThe Martian.'

In [15]:
generate("Recommend me a good science fiction movie and tell me about it.",  tokenizer, model)

"Recommend me a good science fiction movie and tell me about it.\nThe Martian by Andy Weir. It's a great book and movie."

Ответы до и после дообучения похожи, но все-таки отличаются, и каежтся, что дообучение помогает получать более точные ответы.
